In [8]:
#1

import pandas as pd
df=pd.read_csv('data/data_000637.txt', header=0, nrows=15000)
print(df)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0         1     0          123  3869200167        2374        26
1         1     0          124  3869200167        2374        27
2         1     0           63  3869200167        2553        28
3         1     0           64  3869200167        2558        19
4         1     0           64  3869200167        2760        25
...     ...   ...          ...         ...         ...       ...
14995     1     1            4  3869200316        3399         9
14996     1     1           17  3869200316        3400        15
14997     1     1           10  3869200316        3530        16
14998     1     1            8  3869200316        3533        18
14999     1     0          139  3869200316        3539         0

[15000 rows x 6 columns]


In [9]:
#2
maxvalue=df['BX_COUNTER'].max() + 1 
print(maxvalue)

3564


In [3]:
#e3
first_abs=(df['ORBIT_CNT'].iloc[0] * maxvalue + df['BX_COUNTER'].iloc[0] + df['TDC_MEAS'].iloc[0]) * 25
df['ABS_TIME']=((df['ORBIT_CNT'] * maxvalue + df['BX_COUNTER'] + df['TDC_MEAS']) * 25) - first_abs
df.head(10)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
0,1,0,123,3869200167,2374,26,0
1,1,0,124,3869200167,2374,27,25
2,1,0,63,3869200167,2553,28,4525
3,1,0,64,3869200167,2558,19,4425
4,1,0,64,3869200167,2760,25,9625
5,1,0,63,3869200167,2762,4,9150
6,1,0,61,3869200167,2772,14,9650
7,1,0,139,3869200167,2776,0,9400
8,1,0,62,3869200167,2774,21,9875
9,1,0,60,3869200167,2788,7,9875


In [19]:
#Zadanie4
s = pd.to_timedelta(df['ABS_TIME']).max()
print(s)

KeyError: 'ABS_TIME'

In [12]:
#5
df1 = df.groupby('TDC_CHANNEL')['TDC_MEAS'].sum()
df1.sort_values(ascending=False, inplace=True)
df1.head(3)

TDC_CHANNEL
63    11085
64    10889
61     8281
Name: TDC_MEAS, dtype: int64

In [15]:
#6

df=pd.read_csv('data/data_000637.txt', header=0)
non_empty_orbits= df[df['TDC_MEAS'] > 0]['ORBIT_CNT'].nunique()

print(non_empty_orbits)


11001


In [17]:
#7
df=pd.read_csv('data/data_000637.txt', header=0)
unique_orbits = df[df['TDC_CHANNEL'] == 139]['ORBIT_CNT'].nunique()
print(unique_orbits)


10976


In [22]:
#8
df=pd.read_csv('data/data_000637.txt', header=0)
fpga_0 = df[df['FPGA'] == 0]['TDC_CHANNEL'].value_counts()
fpga_1 = df[df['FPGA'] == 1]['TDC_CHANNEL'].value_counts()
print(fpga_0)
print(fpga_1)

FPGA 0 counts: 
TDC_CHANNEL
1       1215
2       1602
3       1493
4       2352
5       1432
       ...  
129        2
130       33
137       32
138       34
139    75617
Length: 124, dtype: int64
FPGA 1 counts: 
TDC_CHANNEL
1      28438
2      32669
3      21970
4      26403
5      15003
       ...  
129       35
130       38
137       36
138       36
139    32442
Length: 132, dtype: int64


TDC_CHANNEL
139    75617
64     64581
63     63724
61     48699
62     48275
       ...  
130       33
137       32
30         4
129        2
39         1
Name: count, Length: 124, dtype: int64
